In [12]:
from __future__ import print_function
import torch as t
from torch.autograd import Variable as V

In [13]:
a = V(t.ones(3,4),requires_grad = True)
a

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)

In [14]:
b = V(t.zeros(3,4))
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [15]:
c = a.add(b)
c

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], grad_fn=<AddBackward0>)

In [22]:
d = c.sum()
d.backward()

In [23]:
a.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [24]:
def f(x):
    y = x**2 * t.exp(x)
    return y
def gradf(x):
    dx = 2*x*t.exp(x) + x**2*t.exp(x)
    return dx

In [37]:
x = V(t.randn(3,4),requires_grad = True)
y = f(x)
y

tensor([[2.6026e-02, 2.2261e+01, 4.3871e-02, 1.5435e-01],
        [1.5825e+01, 1.8026e-01, 3.0477e-03, 2.4850e-01],
        [3.6146e-02, 2.9562e-02, 2.2746e+01, 1.8053e+01]],
       grad_fn=<MulBackward0>)

In [38]:
y.backward(t.ones(y.size()))# 用zeros的话全都是0
x.grad

tensor([[ 0.3738, 46.1876,  0.5046, -0.4558],
        [34.4402, -0.4607,  0.1165, -0.4501],
        [ 0.4510,  0.4019, 47.0572, 38.5522]])

In [39]:
gradf(x)

tensor([[ 0.3738, 46.1876,  0.5046, -0.4558],
        [34.4402, -0.4607,  0.1165, -0.4501],
        [ 0.4510,  0.4019, 47.0572, 38.5522]], grad_fn=<AddBackward0>)

In [40]:
# 看上面的可知，因为y=f(x)，直接y.backward，
# 然后x.grad就可以得到y对x的导数
# 由此可推导，假如z=f(x,y)，那z.backward是可以得到对x y的导数

In [41]:
x = V(t.ones(1))
b = V(t.rand(1),requires_grad = True)
w = V(t.rand(1),requires_grad = True)
y = w * x
z = y + b

In [42]:
z.grad_fn.next_functions

((<MulBackward0 at 0x1a6de87e710>, 0), (<AccumulateGrad at 0x1a6de900550>, 0))

In [43]:
y.grad_fn.next_functions

((<AccumulateGrad at 0x1a6de903ef0>, 0), (None, 0))

In [48]:
z.backward(retain_graph=True)

In [49]:
w.grad

tensor([1.])

In [50]:
z.backward(retain_graph=True)
w.grad

tensor([2.])

In [52]:
def abs(x):
    if x.data[0]>0: return x
    else: return -x
    
x = V(t.ones(1),requires_grad=True)
y = abs(x)
y.backward()
x.grad

tensor([1.])

In [58]:
def f(x):
    result = 1
    for ii in x:
        if ii.data>0: result=ii*result# data或item()都可以，data[0]报错
    return result
x = V(t.arange(-2,4).float(),requires_grad=True)
y = f(x) # y = x[3]*x[4]*x[5]
y

tensor(6., grad_fn=<MulBackward0>)

In [59]:
y.backward()
x.grad

tensor([0., 0., 0., 6., 3., 2.])

In [65]:
from torch.autograd import Function

class Sigmoid(Function):
    @staticmethod
    def forward(ctx,x):
        output = 1/(1+t.exp(-x))
        ctx.save_for_backward(output)
        return output
    
    @staticmethod
    def backward(ctx,grad_output):
        output, = ctx.saved_tensors
        grad_x = output * (1-output)*grad_output
        return grad_x

In [63]:
test_input = V(t.randn(3,4),requires_grad=True)
t.autograd.gradcheck(Sigmoid.apply,(test_input),eps=1e-3)

C:\Users\95854\Anaconda3\lib\site-packages\torch\autograd\gradcheck.py:239: UserWarning: At least one of the inputs that requires gradient is not of double precision floating point. This check will likely fail if all the inputs are not of double precision floating point. 
  'At least one of the inputs that requires gradient '
C:\Users\95854\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  if sys.path[0] == '':


True